In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import re
import string


#url_testDATA='https://drive.google.com/file/d/1q8_G19zYuD4C2l8u4CrGe_yiitjI1h2I/view?usp=sharing'
url_trainDATA='https://drive.google.com/file/d/1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW/view?usp=sharing'

#https://www.kaggle.com/datatattle/email-classification-nlp?select=SMS_train.csv
url=url_trainDATA
file_id=url.split('/')[-2]
print('LOG file id = ',file_id)
dwn_url='https://drive.google.com/uc?id=' + file_id
print('LOG download url = ', dwn_url)

df = pd.read_csv(dwn_url, encoding = "ISO-8859-1")

LOG file id =  1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW
LOG download url =  https://drive.google.com/uc?id=1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW


In [3]:
df.drop(['S. No.'], axis = 1, inplace = True)
print(df)

                                          Message_body     Label
0                           Rofl. Its true to its name  Non-Spam
1    The guy did some bitching but I acted like i'd...  Non-Spam
2    Pity, * was in mood for that. So...any other s...  Non-Spam
3                 Will ü b going to esplanade fr home?  Non-Spam
4    This is the 2nd time we have tried 2 contact u...      Spam
..                                                 ...       ...
952  hows my favourite person today? r u workin har...  Non-Spam
953                      How much you got for cleaning  Non-Spam
954  Sorry da. I gone mad so many pending works wha...  Non-Spam
955                                 Wat time ü finish?  Non-Spam
956                    Just glad to be talking to you.  Non-Spam

[957 rows x 2 columns]


In [4]:
max_seq_len = 140
max_features = 30000
stop_words = stopwords.words("english")

def digital_label(text):
    if text == 'Non-Spam':
        return 1.0
    elif text == 'Spam':
        return 0.0
    
def clean_text(text):
    text = text.lower()
    text = re.sub("@\S+", " ", text)  
    text = re.sub("https*\S+", " ", text)
    text = re.sub("www\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', ' ', text)        
    text = re.sub('\s{2,}',' ', text)
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

def convert(text):
    tok = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(t) for t in tok]
    res = [dictionary.index(i) for i in lem]
    return res

In [5]:
df.Message_body = df.Message_body.apply(clean_text)
print (df)

                                          Message_body     Label
0                                       rofl true name  Non-Spam
1    guy bitching acted like interested buying some...  Non-Spam
2                               pity mood suggestions   Non-Spam
3                         ü b going esplanade fr home   Non-Spam
4    nd time tried contact u u £ pound prize claim ...      Spam
..                                                 ...       ...
952  hows favourite person today r u workin hard sl...  Non-Spam
953                                  much got cleaning  Non-Spam
954              sorry da gone mad many pending works   Non-Spam
955                                 wat time ü finish   Non-Spam
956                                      glad talking   Non-Spam

[957 rows x 2 columns]


In [6]:
df.Label = df.Label.apply(digital_label)
print(df)

                                          Message_body  Label
0                                       rofl true name    1.0
1    guy bitching acted like interested buying some...    1.0
2                               pity mood suggestions     1.0
3                         ü b going esplanade fr home     1.0
4    nd time tried contact u u £ pound prize claim ...    0.0
..                                                 ...    ...
952  hows favourite person today r u workin hard sl...    1.0
953                                  much got cleaning    1.0
954              sorry da gone mad many pending works     1.0
955                                 wat time ü finish     1.0
956                                      glad talking     1.0

[957 rows x 2 columns]


In [7]:
X_train = df.Message_body
Y_train = df.Label
print("\nmessage:\n", X_train)
print("\nlabel:\n", Y_train)


message:
 0                                         rofl true name
1      guy bitching acted like interested buying some...
2                                 pity mood suggestions 
3                           ü b going esplanade fr home 
4      nd time tried contact u u £ pound prize claim ...
                             ...                        
952    hows favourite person today r u workin hard sl...
953                                    much got cleaning
954                sorry da gone mad many pending works 
955                                   wat time ü finish 
956                                        glad talking 
Name: Message_body, Length: 957, dtype: object

label:
 0      1.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
952    1.0
953    1.0
954    1.0
955    1.0
956    1.0
Name: Label, Length: 957, dtype: float64


In [8]:
whole_text = ""
for i in X_train:
    whole_text += i + ' '
tokens = word_tokenize(whole_text)
lemmatizer = WordNetLemmatizer()
lemtok = [lemmatizer.lemmatize(t) for t in tokens]
dictionary = list(set(lemtok))


In [9]:
X_train = X_train.apply(convert)
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen = max_seq_len)
y_train = np.asarray(Y_train).astype("float32")
print("\nconvert\n", X_train)
print("\nsequences\n", x_train)
print("\nmassiv\n", y_train)


convert
 0                                       [985, 1267, 413]
1      [1037, 999, 1360, 2035, 469, 1471, 864, 428, 1...
2                                       [2263, 841, 890]
3                       [984, 2000, 742, 997, 862, 1287]
4      [2272, 82, 1109, 2317, 1209, 1209, 2350, 1765,...
                             ...                        
952    [1889, 1141, 91, 2314, 2543, 1209, 1395, 788, ...
953                                    [265, 1504, 2455]
954              [2361, 2347, 657, 503, 2307, 2395, 962]
955                                  [419, 82, 984, 313]
956                                         [1118, 1132]
Name: Message_body, Length: 957, dtype: object

sequences
 [[   0    0    0 ...  985 1267  413]
 [   0    0    0 ...  362 1209  518]
 [   0    0    0 ... 2263  841  890]
 ...
 [   0    0    0 ... 2307 2395  962]
 [   0    0    0 ...   82  984  313]
 [   0    0    0 ...    0 1118 1132]]

massiv
 [1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [10]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [11]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [15]:
embed_dim = 20  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 14  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_seq_len,))
embedding_layer = TokenAndPositionEmbedding(max_seq_len, max_features, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=10, epochs=6, validation_data=(x_train, y_train)
)

Epoch 1/6
96/96 [==============================] - 7s 49ms/step - loss: 0.4105 - accuracy: 0.8600 - val_loss: 0.3574 - val_accuracy: 0.8725
Epoch 2/6
96/96 [==============================] - 4s 47ms/step - loss: 0.3681 - accuracy: 0.8704 - val_loss: 0.3303 - val_accuracy: 0.8725
Epoch 3/6
96/96 [==============================] - 4s 44ms/step - loss: 0.3369 - accuracy: 0.8662 - val_loss: 0.2917 - val_accuracy: 0.8725
Epoch 4/6
96/96 [==============================] - 4s 42ms/step - loss: 0.1483 - accuracy: 0.9457 - val_loss: 0.0360 - val_accuracy: 0.9864
Epoch 5/6
96/96 [==============================] - 4s 43ms/step - loss: 0.0479 - accuracy: 0.9864 - val_loss: 0.0162 - val_accuracy: 0.9948
Epoch 6/6
96/96 [==============================] - 4s 43ms/step - loss: 0.0232 - accuracy: 0.9937 - val_loss: 0.0062 - val_accuracy: 0.9979


In [17]:
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.006241454277187586
Test accuracy: 0.9979101419448853
